In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from antipark import *
from antipark.models import Product, Category

2016-12-20 08:11:58,731 INFO sqlalchemy.engine.base.Engine SELECT services.id AS services_id, services.name AS services_name, services.image AS services_image, services.text AS services_text 
FROM services
2016-12-20 08:11:58,734 INFO sqlalchemy.engine.base.Engine {}


In [12]:
products = Product.query.all()
categories = Category.query.all()

2016-12-20 08:18:25,335 INFO sqlalchemy.engine.base.Engine SELECT products.id AS products_id, products.category AS products_category, products.subcategory AS products_subcategory, products.title AS products_title, products.attr AS products_attr, products.specs AS products_specs, products.price AS products_price, products.stage_price AS products_stage_price, products.image AS products_image 
FROM products
2016-12-20 08:18:25,338 INFO sqlalchemy.engine.base.Engine {}
2016-12-20 08:18:25,368 INFO sqlalchemy.engine.base.Engine SELECT categories.id AS categories_id, categories.category AS categories_category, categories.description AS categories_description 
FROM categories
2016-12-20 08:18:25,370 INFO sqlalchemy.engine.base.Engine {}


In [10]:
import json

with open('../antipark/db.json', encoding='utf=8', mode='r') as fo:
    data = json.load(fo)

d = {}
for k,v in data['products'].items():
    d[k] = v

In [ ]:
# if smth was wrong
# db.session.rollback()

for category, values in d.items():
    
    description = values['desc']
    
    cat = Category(category=category,
                   description=description,
                  )
    db.session.add(cat)
    db.session.commit()

categories = Category.query.all()
categories

In [13]:
from collections import namedtuple

Cat = namedtuple('Cat', ['category', 'idx'])
cats = []
for category in categories:
    if category.category == 'Знаки дорожные':
#         flag_cat = Cat(category.category, category.id)
        continue
    cat = Cat(category.category, category.id)
    cats.append(cat)
    
cats

[Cat(category='Столбики парковочные', idx=3),
 Cat(category='Конусы', idx=4),
 Cat(category='Бетонные урны', idx=5),
 Cat(category='Автоматические шлагбаумы', idx=6),
 Cat(category='Пластиковые сетки', idx=7),
 Cat(category='Бетонные полусферы', idx=8),
 Cat(category='Барьеры', idx=9),
 Cat(category='Турникеты', idx=10),
 Cat(category='Лежачий полицейский', idx=11),
 Cat(category='Бетонные вазоны', idx=12),
 Cat(category='Бетонные скамейки', idx=13),
 Cat(category='Оградительная лента', idx=14),
 Cat(category='Приводы для ворот', idx=15),
 Cat(category='Флагштоки', idx=17)]

In [ ]:
for category, values in d.items():
    
    category_id = None
    for cat in cats:
        if category == cat.category:
            category_id = cat.idx
            break
    if not category_id:
        continue
    
    all_vars = values['variations']
    
    for var in all_vars:
        # prices
        prices = all_vars[var]['prices'].strip('[]').split(', ')
            
        # stage_prices
        stage_prices = all_vars[var]['stage'].strip('[]').split(', ')
            
        # names
        names = all_vars[var]['names'].strip('[]').split(', ')
        # attrs
        attrs = all_vars[var]['attrs'].strip('[]').split(', ')
        
        # subcategory
        subcategory = all_vars[var]['general']
        
        # specs
        chars = all_vars[var]['chars'].strip('[]').split(', ')
        char_values_str = all_vars[var]['values'].split('], [')
        char_values_temp = []
        for ch_v in char_values_str:
            char_values_temp.append(ch_v.strip('[]').split(', '))
                
        specs = []
        print(chars)
        for ch_v in char_values_temp:
            print(ch_v)
            print()
            char_values_dict = {}
            for i, ch in enumerate(ch_v):
                char_values_dict[chars[i]] = ch
            specs.append(str(char_values_dict))
            
        # images`
        images_str = all_vars[var]['imgs'].strip('[]').split(', ')
        images = []
        for img in images_str:
            image = all_vars[var]['dest'] + img + ".jpg"
            images.append(image)
            
        
        # commit rec to db
        for i, title in enumerate(names):
            prod = Product(category=category_id,
                           subcategory=subcategory,
                           title=title,
                           attr=attrs[i],
                           specs=specs[i],
                           price=prices[i],
                           stage_price=stage_prices[i],
                           image=images[i],
                           )
            db.session.add(prod)
            db.session.commit()

['Высота', 'Диаметр', 'Материал', 'Цвет']
['1200 мм', '40 мм', 'Пластик', 'Оранжевый']

2016-12-20 08:18:52,669 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-12-20 08:18:52,671 INFO sqlalchemy.engine.base.Engine INSERT INTO products (category, subcategory, title, attr, specs, price, stage_price, image) VALUES (%(category)s, %(subcategory)s, %(title)s, %(attr)s, %(specs)s, %(price)s, %(stage_price)s, %(image)s)
2016-12-20 08:18:52,672 INFO sqlalchemy.engine.base.Engine {'title': 'Веха сигнальная', 'specs': "{'Цвет': 'Оранжевый', 'Высота': '1200 мм', 'Материал': 'Пластик', 'Диаметр': '40 мм'}", 'stage_price': '-', 'subcategory': '', 'price': '149', 'attr': 'Веха сигнальная ВС.1', 'image': 'setki/veha.jpg', 'category': 7}
2016-12-20 08:18:52,695 INFO sqlalchemy.engine.base.Engine COMMIT
['Размер листа', 'Материал', 'Цвет']
['1х1 м', 'Полипропилен', 'Тёмно-зелёный']

2016-12-20 08:18:52,738 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-12-20 08:18:52,739 INFO sqlalche

In [15]:
for category in categories:
    if category.category != 'Знаки дорожные':
        continue
    cat_znaki = Cat(category.category, category.id)
    
cat_znaki

Cat(category='Знаки дорожные', idx=16)

In [16]:
znaki = []
znak = namedtuple('Znak', ['title', 'attr', 'specs', 'price', 'image'])

In [18]:
for category, values in d.items():
    if category != 'Знаки дорожные':
        continue
    znak_dict = values['variations']['var1']
znak_dict

{'attrs': '[1.1-1.2, 1.5-1.33, 2.3.1-2.3.7, 2.4, 2.1, 2.2, 2.7, 5.5, 5.6, 5.8, 5.9-5.12, 5.19, 5.20, 6.2-6.8.3, 8.13, 2.6, 3.1-3.33, 4.1.1-4.7, Остановки 5.16-5.18, Жилая зона 5.21, 5.22, Зона с ограничением стоянки 5.27-5.34, Километровый знак тип 6.13, тип 2.5, тип 7.1-7.18, тип 5.15.1, 5.15.7, Таблички Зона действия и т.д. 8.1.1-8.1.2, 8.1.3-8.21.3, Выезд на дорогу с односторонним движением тип 5.7.1-5.7.2. Схема объезда 6.17, Однопутная железная дорога тип 1.3.1, Многопутная железная дорога, тип 1.3.2, Направление поворота, тип 1.34.1-1.34.2, Направление поворота, тип 1.34.1-1.34.3, Направление поворота, тип 1.34-1.34.4, Препятствие, тип 8.22.1-8.22.3, Указатель перестроения на другую проезжую часть 6.19.1-6.19.2, Щиты индивидуального проетирования, , , ]',
 'chars': '[Размер, Цена. Маска, Цена. Тип А (5 лет), Цена. Тип А (7 лет), Цена. Тип Б (10 лет)]',
 'dest': 'znaki/',
 'general': '',
 'imgs': '[triangle, rectangle, circle, zona-deistviya, 5_16-18_21-22_27-34, 2_5, 7_1-18, 5_15

In [23]:
# names
zn_names = znak_dict['names'].strip('[]').split(', ')
print(len(zn_names))
print(zn_names)

21
['Дорожные знаки треугольные', 'Дорожные знаки квадратные', 'Дорожные знаки круглые', 'Дорожные знаки', 'Дорожные знаки таблички', 'Дорожные знаки СТОП', 'Дорожные знаки сервиса', 'Дорожные знаки движения по полосам', 'Дорожные знаки таблички', 'Дорожные знаки выезд', 'Дорожные знаки ЖД', 'Дорожные знаки ЖД', 'Дорожные знаки поворота', 'Дорожные знаки поворота', 'Дорожные знаки поворота', 'Дорожные знаки препятствия', 'Дорожные знаки указатели', 'Дорожные знаки щиты', 'Фундамент дорожного знака', 'Стойка дорожного знака', 'Крепеж дорожного знака']


In [24]:
# attrs
zn_attrs = znak_dict['attrs'].strip('[]').split(', ')
print(len(zn_attrs))
print(zn_attrs)

46
['1.1-1.2', '1.5-1.33', '2.3.1-2.3.7', '2.4', '2.1', '2.2', '2.7', '5.5', '5.6', '5.8', '5.9-5.12', '5.19', '5.20', '6.2-6.8.3', '8.13', '2.6', '3.1-3.33', '4.1.1-4.7', 'Остановки 5.16-5.18', 'Жилая зона 5.21', '5.22', 'Зона с ограничением стоянки 5.27-5.34', 'Километровый знак тип 6.13', 'тип 2.5', 'тип 7.1-7.18', 'тип 5.15.1', '5.15.7', 'Таблички Зона действия и т.д. 8.1.1-8.1.2', '8.1.3-8.21.3', 'Выезд на дорогу с односторонним движением тип 5.7.1-5.7.2. Схема объезда 6.17', 'Однопутная железная дорога тип 1.3.1', 'Многопутная железная дорога', 'тип 1.3.2', 'Направление поворота', 'тип 1.34.1-1.34.2', 'Направление поворота', 'тип 1.34.1-1.34.3', 'Направление поворота', 'тип 1.34-1.34.4', 'Препятствие', 'тип 8.22.1-8.22.3', 'Указатель перестроения на другую проезжую часть 6.19.1-6.19.2', 'Щиты индивидуального проетирования', '', '', '']


In [73]:
# specs
chars = znak_dict['chars'].strip('[]').split(', ')
char_values_str = znak_dict['values'].split('], [')
char_values_temp = []
for ch_v in char_values_str:
    char_values_temp.append(ch_v.strip('[]').split(', '))
    
print(len(char_values_temp))
print(char_values_temp)

21
[['900 мм', '399р', '799р', '850р', '1350р'], ['700х700 мм', '450р', '899р', '1050р', '1499р'], ['Ф700 мм', '450р', '899р', '1050р', '1499р'], ['600х900 мм', '499р', '999р', '1099р', '1699р'], ['300х200 мм', '199р', '399р', '499р', '699р'], ['700 мм', '499р', '949р', '999р', '1599р'], ['1050х700 мм', '699р', '1299р', '1399р', '2299р'], ['1400х700 мм', '899р', '1799р', '1899р', '3199р'], ['700х350 мм', '299р', '549р', '599р', '799р'], ['1050х350 мм', '399р', '749р', '799р', '1299р'], ['-', '-', '899р', '999р', '1499р'], ['-', '-', '1299р', '1399р', '2299р'], ['500х615 мм', '349р', '640р', '699р', '1099р'], ['500х1160 мм', '699р', '1299р', '1399р', '2399р'], ['500х2250 мм', '899р', '3899р', '3999р', '5599р'], ['500х1700 мм', '799р', '1499р', '1599р', '2699р'], ['1500х2000 мм', '-', '-', '-', '-'], ['За один м2', '-', '3299р', ' 3599р', '4999р'], ['500х500х300 мм', '599р', '-', '-', '-'], ['Ф76 4000 мм', '1799р', '-', '-', '-'], ['За комплект', '250р', '-', '-', '-']]


In [41]:
chars_imp = [x.split('. ')[1] for x in chars]
chars_imp

['Маска', 'Тип А (5 лет)', 'Тип А (7 лет)', 'Тип Б (10 лет)']

In [32]:
specs_char = ['Размер', 'Тип', 'Цена']
specs_char

['Размер', 'Тип', 'Цена']

In [74]:
spec_values = []
for value_list in char_values_temp:
    spec_dict = {}
    size = value_list.pop(0)
    spec_dict["Размер"] = size
    for i, value in enumerate(value_list):
        spec_dict['Тип'] = chars_imp[i]
        spec_dict['Цена'] = value
        new_dict = dict(**spec_dict)
        spec_values.append(new_dict)

In [76]:
values = []
for spec in spec_values:
    print(spec)
    values.append(spec.pop('Цена'))

{'Тип': 'Маска', 'Цена': '399р', 'Размер': '900 мм'}
{'Тип': 'Тип А (5 лет)', 'Цена': '799р', 'Размер': '900 мм'}
{'Тип': 'Тип А (7 лет)', 'Цена': '850р', 'Размер': '900 мм'}
{'Тип': 'Тип Б (10 лет)', 'Цена': '1350р', 'Размер': '900 мм'}
{'Тип': 'Маска', 'Цена': '450р', 'Размер': '700х700 мм'}
{'Тип': 'Тип А (5 лет)', 'Цена': '899р', 'Размер': '700х700 мм'}
{'Тип': 'Тип А (7 лет)', 'Цена': '1050р', 'Размер': '700х700 мм'}
{'Тип': 'Тип Б (10 лет)', 'Цена': '1499р', 'Размер': '700х700 мм'}
{'Тип': 'Маска', 'Цена': '450р', 'Размер': 'Ф700 мм'}
{'Тип': 'Тип А (5 лет)', 'Цена': '899р', 'Размер': 'Ф700 мм'}
{'Тип': 'Тип А (7 лет)', 'Цена': '1050р', 'Размер': 'Ф700 мм'}
{'Тип': 'Тип Б (10 лет)', 'Цена': '1499р', 'Размер': 'Ф700 мм'}
{'Тип': 'Маска', 'Цена': '499р', 'Размер': '600х900 мм'}
{'Тип': 'Тип А (5 лет)', 'Цена': '999р', 'Размер': '600х900 мм'}
{'Тип': 'Тип А (7 лет)', 'Цена': '1099р', 'Размер': '600х900 мм'}
{'Тип': 'Тип Б (10 лет)', 'Цена': '1699р', 'Размер': '600х900 мм'}
{'Тип': '

In [48]:
# images
images_str = znak_dict['imgs'].strip('[]').split(', ')
images = []
for img in images_str:
    image = znak_dict['dest'] + img + ".jpg"
    images.append(image)
    
len(images)

21

In [81]:
for i, title in enumerate(zn_names):
    for j in range(len(chars_imp)):
        prod = Product(category=cat_znaki.idx,
                       title=title,
                       specs=str(spec_values[i*4 + j]),
                       price=values[i*4 + j],
                       image=images[i],
                       stage_price='-',
                       )
        db.session.add(prod)
        db.session.commit()

2016-12-20 09:24:49,324 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-12-20 09:24:49,326 INFO sqlalchemy.engine.base.Engine INSERT INTO products (category, subcategory, title, attr, specs, price, stage_price, image) VALUES (%(category)s, %(subcategory)s, %(title)s, %(attr)s, %(specs)s, %(price)s, %(stage_price)s, %(image)s)
2016-12-20 09:24:49,327 INFO sqlalchemy.engine.base.Engine {'title': 'Дорожные знаки треугольные', 'specs': "{'Тип': 'Маска', 'Размер': '900 мм'}", 'stage_price': '-', 'subcategory': None, 'price': '399р', 'attr': None, 'image': 'znaki/triangle.jpg', 'category': 16}
2016-12-20 09:24:49,356 INFO sqlalchemy.engine.base.Engine COMMIT
2016-12-20 09:24:49,391 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-12-20 09:24:49,393 INFO sqlalchemy.engine.base.Engine INSERT INTO products (category, subcategory, title, attr, specs, price, stage_price, image) VALUES (%(category)s, %(subcategory)s, %(title)s, %(attr)s, %(specs)s, %(price)s, %(stage_price)s, %(im

In [ ]:
#         # prices
#         prices = all_vars[var]['prices'].strip('[]').split(', ')

#         # stage_prices
#         stage_prices = all_vars[var]['stage'].strip('[]').split(', ')



# subcategory
subcategory = all_vars[var]['general']

# specs
chars = all_vars[var]['chars'].strip('[]').split(', ')
char_values_str = all_vars[var]['values'].split('], [')
char_values_temp = []
for ch_v in char_values_str:
    char_values_temp.append(ch_v.strip('[]').split(', '))

for 

# values
values = all_vars[var]['values'].strip('[]').split(', ')


char_values_str = all_vars[var]['values'].split('], [')
char_values_temp = []
for ch_v in char_values_str:
    char_values_temp.append(ch_v.strip('[]').split(', '))

specs = []
print(chars)
for ch_v in char_values_temp:
    print(ch_v)
    print()
    char_values_dict = {}
    for i, ch in enumerate(ch_v):
        char_values_dict[chars[i]] = ch
    specs.append(str(char_values_dict))

# images
images_str = all_vars[var]['imgs'].strip('[]').split(', ')
images = []
for img in images_str:
    image = all_vars[var]['dest'] + img + ".jpg"
    images.append(image)




# commit rec to db
#         for i, title in enumerate(names):
#             prod = Product(category=category_id,
#                            subcategory=subcategory,
#                            title=title,
#                            specs=specs[i],
#                            price=prices[i],
#                            stage_price=stage_prices[i],
#                            image=images[i],
#                            )
#             db.session.add(prod)
#             db.session.commit()